In [2]:
import numpy as np
import pandas as pd

In [4]:
df_schema = pd.read_csv("stack-overflow-developer-survey-2017/survey_results_schema.csv")

In [17]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 2000)

In [18]:
df_schema.head(145)

,Column,Question
0,Respondent,Respondent ID number
1,Professional,Which of the following best describes you?
2,ProgramHobby,Do you program as a hobby or contribute to open source projects?
3,Country,In which country do you currently live?
4,University,"Are you currently enrolled in a formal, degree-granting college or university program?"
5,EmploymentStatus,Which of the following best describes your current employment status?
6,FormalEducation,Which of the following best describes the highest level of formal education that you've completed?
7,MajorUndergrad,Which of the following best describes your main field of study (aka 'major') in college or university/for your undergraduate studies?
8,HomeRemote,How often do you work from home or remotely?
9,CompanySize,"In terms of the number of employees, how large is the company or organization you work for?"
